In [17]:
import os
import pandas as pd
import pickle
from ast import literal_eval
import logging
import argparse
from simpletransformers.ner import (
    NERArgs,
    NERModel,
)

In [2]:
BIO_type = 1

labelset = ["B-p", "I-p", "B-n", "I-n", "O"]
type_ls = ['-p', '-n']

In [3]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

pos = pd.read_csv('pos_sent.csv')
print(len(pos))
col_name = pos.columns[6+BIO_type]
pos = pos.dropna(axis=0, subset=[col_name])
print(len(pos))

3149
3149


In [4]:
data = []
for i in range(len(pos)):
    words = pos.iloc[i, 1].split(' ')
    tags = literal_eval(pos.iloc[i, 6+BIO_type])
    for j in range(len(words)):
        data.append([i, words[j], tags[j]])
df = pd.DataFrame(data, columns=['sentence_id', 'words', 'labels'])

In [5]:
model_args = NERArgs()
model_args.reprocess_input_data = True
model_args.save_model_every_epoch = False
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.manual_seed = 1
model_args.use_multiprocessing = True
model_args.do_lower_case = False

In [6]:
def phrase_F1(ref, pred):
    return 0

# Create a TransformerModel
model = NERModel(
    "xlmroberta",
    "../ner/Noutputs/best_model",
    labels=labelset,
    args=model_args,
)

In [7]:
result, model_outputs, pred_label = model.eval_model(df, F1_score=phrase_F1)
# 'pred_label' is a list, where each element is the list of predicted labels for one sentence

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/3149 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/394 [00:00<?, ?it/s]

/opt/conda/envs/hl/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
INFO:simpletransformers.ner.ner_model:{'eval_loss': 2.7530998868990673, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'F1_score': 0}


In [20]:
p=[str(l) for l in pred_label]

In [21]:
pos = pd.read_csv('pos_sent.csv')
pos['BIO_1']=p

In [22]:
pos

,idx,text,main_heading,heading,topic,paper_idx,BIO,BIO_1,offset1,pro1,offset2,pro2,offset3,pro3,mask,labels,title,paper
0,2,Recurrent Neural Network Grammars,title,title,constituency_parsing,0,"['O', 'O', 'O', 'O']","['B-n', 'I-n', 'I-n', 'I-n']",1,0.000000,1,0.004484,1,0.000000,1,research-problem,title,constituency_parsing0
1,12,"In this paper , we introduce recurrent neural ...",Introduction,Introduction,constituency_parsing,0,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'O', 'O', 'O', 'O', 'B-p', 'B-n', 'I-n',...",4,0.043478,11,0.049327,4,0.200000,1,model,Introduction,constituency_parsing0
2,15,"We give two variants of the algorithm , one fo...",Introduction,Introduction,constituency_parsing,0,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'B-p', 'B-n', 'I-n', 'B-p', 'O', 'B-n', ...",7,0.076087,14,0.062780,7,0.350000,1,model,Introduction,constituency_parsing0
3,20,Similar to previously published discriminative...,Introduction,Introduction,constituency_parsing,0,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",12,0.130435,19,0.085202,12,0.600000,1,model,Introduction,constituency_parsing0
4,25,We present a simple importance sampling algori...,Introduction,Refer to for an example .,constituency_parsing,0,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'B-p', 'O', 'B-n', 'I-n', 'I-n', 'I-n', ...",17,0.184783,24,0.107623,17,0.850000,1,model,Introduction: Refer to for an example .,constituency_parsing0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3144,17,The key insights in our approach are 1 . to jo...,Introduction,Introduction,natural_language_inference,9,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",9,0.250000,16,0.246154,9,0.818182,1,approach,Introduction,natural_language_inference9
3145,18,We refer to our model as BERT joint to emphasi...,Introduction,Introduction,natural_language_inference,9,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'O', 'O', 'O', 'O', 'B-p', 'B-n', 'I-n',...",10,0.277778,17,0.261538,10,0.909091,1,approach,Introduction,natural_language_inference9
3146,54,We initialized our model from a BERT model alr...,Experiments,Experiments,natural_language_inference,9,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'B-p', 'B-n', 'I-n', 'B-p', 'O', 'B-n', ...",1,0.111111,53,0.815385,1,0.111111,1,hyperparameters,Experiments,natural_language_inference9
3147,56,We trained the model by minimizing loss L from...,Experiments,Experiments,natural_language_inference,9,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'B-p', 'O', 'B-n', 'B-p', 'B-n', 'B-n', ...",3,0.333333,55,0.846154,3,0.333333,1,hyperparameters,Experiments,natural_language_inference9


In [ ]:
type_ls = ['-p', '-n']
def get_entity_spans(idx, ls):
    # given a sequence of BIO taggs, get the list of tuples representing spans of entities
    spans = [[],[]]
    for t in range(2):
        st, ed = 0, 0
        for i in range(len(ls)):
            #st, ed = 0, 0
            if ls[i] == 'B' + type_ls[t]:
                st, ed = i, i + 1
                for j in range(i+1, len(ls)):
                    if ls[j] == 'I' + type_ls[t]:
                        ed += 1
                    else:
                        break
                phrase=' '.join(pos.loc[idx,'text'].split(' ')[st:ed])
                tup=(phrase, (st, ed))
                spans[t].append(tup)
    spans[0] = sorted(spans[0], key=lambda x: x[0])
    spans[0] = sorted(spans[1], key=lambda x: x[0])
    return spans

In [ ]:
def get_entity_spans(k, ls):
    spans = [[],[]]
    for i in range(len(ls)):
        if ls[i]=='B-p':
            for j in range(i+1,len(ls)):
                if ls[j]!='I-p':
                    phrase=' '.join(pos.loc[k,'text'].split(' ')[i:j])
                    tup=(phrase, (i, j))
                    spans[0].append(tup)
                    break
        elif ls[i]=='B-n':
            for j in range(i+1,len(ls)):
                if ls[j]!='I-n':
                    phrase=' '.join(pos.loc[k,'text'].split(' ')[i:j])
                    tup=(phrase, (i, j))
                    spans[1].append(tup)
                    break
    return spans
                    

In [ ]:
all=[]
for k in range(len(pred_label)):
    all.append(get_entity_spans(k, pred_label[k]))
all

In [ ]:
all=[]
for k in range(len(pred_label)):
    all.append(get_entity_spans(k, pred_label[k]))
all


In [ ]:
dt=pd.read_csv('pos_sent.csv')
#dt=dt['labels','text']
a=pd.DataFrame(all)
a.columns=['predicates', 'subj/obj']
a=pd.concat([dt[['labels','text']],a],axis=1)
a['triple_A']='[]'
a['triple_B']='[]'
a['triple_C']='[]'
a['triple_D']='[]'
a=a.reset_index(drop=True)

In [ ]:
a.to_csv('triples.csv',index=False)